In [ ]:
import ipyvuetify as v
import ipywidgets
from scripts import run_aoi_selector as aoi
from datetime import datetime
from utils import utils 
from utils import widget_manager as wm
from utils import messages as ms

In [ ]:
#input list
file_input = None
file_name = 'Manual_{0}'.format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
country_selection = None
drawing_method = None
drawn_feat = None

In [ ]:
#output list 
asset_name = None

In [ ]:
aoi_output = v.Alert(children=[ms.AOI_MESSAGE], type='info', text=True, class_="mt-5")

In [ ]:
#map widget
from scripts import mapping
import ee

ee.Initialize()

dc, m = mapping.init_map()

In [ ]:
#import file
aoi_file_input = v.TextField(label='select a file', v_model=None, class_="d-none")
def on_file_input_change(widget, event, data):
    global file_input
    file_input = widget.v_model

aoi_file_input.on_event('change', on_file_input_change )

aoi_file_name = v.TextField(label='select a filename', v_model=file_name, class_="d-none")
def on_file_name_change(widget, event, data):
    global file_name
    file_name = widget.v_model

aoi_file_name.on_event('change', on_file_name_change )

In [ ]:
#country boundaries selector 
from utils import utils

fips_disc = utils.create_FIPS_dic()
aoi_country_selection = v.Select(items=[*fips_disc], label='Country/Province', v_model=None, class_="d-none")
def on_country_change(widget, event, data):
    global country_selection
    global aoi_output

    country_selection = widget.v_model
    wm.displayIO(aoi_output, 'info', 'country selected: {0}'.format(str(widget.v_model)))

aoi_country_selection.on_event('change', on_country_change )

In [ ]:
#GEE asset
aoi_asset_name = v.TextField(label='select a GEE asset', class_="d-none", v_model=None)

def on_asset_change(widget, event, data):
    global asset_name
    asset_name = widget.v_model
    
aoi_asset_name.on_event('change', on_asset_change)

In [ ]:
# add a method selector 
selection_method = [
    'Country boundaries', 
    'draw a shape', 
    'Upload file', 
    'use GEE asset'
]
 
aoi_drawing_method = v.Select(items=selection_method, label='AOI selection method', v_model=None)

def toggle_inputs(input_list):
    global aoi_file_input
    global aoi_file_name
    global aoi_country_selection
    global aoi_asset_name
    
    widget_list = [aoi_file_input, aoi_file_name, aoi_country_selection, aoi_asset_name]
    
    for input_item in widget_list:
        if input_item in input_list:
            input_item.class_ = 'd-inline'
        else:
            input_item.class_ = 'd-none'
        
    
def on_method_change(widget, event, data):

    global aoi_file_input
    global aoi_file_name
    global aoi_country_selection
    global aoi_asset_name
    global drawing_method
    
    drawing_method = widget.v_model
    
    try:
        m.add_control(dc)
    except:
        pass
    dc.clear()
    m.remove_control(dc)
    if drawing_method == selection_method[0]:
        toggle_inputs([aoi_country_selection])
    elif drawing_method == selection_method[1]:
        toggle_inputs([aoi_file_name])
        m.add_control(dc)
    elif drawing_method == selection_method[2]:
        toggle_inputs([aoi_file_input])
    elif drawing_method == selection_method[3]:
        toggle_inputs([aoi_asset_name])

aoi_drawing_method.on_event('change', on_method_change)

In [ ]:
import geemap

# Handle draw events
def handle_draw(self, action, geo_json):
    
    global drawn_feat
    global aoi_output

    geom = geemap.geojson_to_ee(geo_json, False)
    feature = ee.Feature(geom)
    drawn_feat = ee.FeatureCollection(feature) 
    
dc.on_draw(handle_draw)

In [ ]:
#validate the selected data 
aoi_select_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-map-marker-check']),
        'Select these inputs'
    ]
)

def aoi_on_click(widget, event, data):
    
    global widget_alert
    global drawing_method
    global asset_name
    global m
    
    #io variables 
    global file_input
    global file_name
    global country_selection
    global drawing_method 
    global asset_name
    global year
    global selection_method
    global drawn_feat
    
    #do something 
    wm.toggleLoading(aoi_select_btn)
    asset_name = aoi.run_aoi_selctor(
        file_input=file_input, 
        file_name=file_name, 
        country_selection=country_selection, 
        asset_name=asset_name, 
        drawing_method=drawing_method,
        widget_alert=aoi_output,
        list_method=selection_method,
        m=m,
        drawn_feat=drawn_feat
    )
    wm.toggleLoading(aoi_select_btn)
    
    #update the map with asset
    mapping.update_map(m, dc, asset_name)

aoi_select_btn.on_event('click', aoi_on_click)

In [ ]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    class_="pa-5",
    row=True,
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[aoi_drawing_method]),
        v.Flex(xs12=True, children=[aoi_country_selection]),
        v.Flex(xs12=True, children=[aoi_file_input]),
        v.Flex(xs12=True, children=[aoi_file_name]),
        v.Flex(xs12=True, children=[aoi_asset_name]),
        v.Flex(xs12=True, children=[aoi_select_btn]),
        v.Flex(xs12=True, children=[aoi_output]),
    ]
)
#inputs

AOI_content_main =  v.Layout(
    dark=True,
    _metadata={'mount_id': 'aoi-selector-widget'},
    xs12=True,
    row=True,
    class_="ma-5 d-inline",
    children=[
        v.Card(
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Step 1: AOI selection']),
                v.Layout(
                    row=True,
                    xs12=True,
                    children=[
                        v.Flex(xs12=True, md6=True, children=[inputs]),
                        v.Flex(class_="pa-5", xs12=True, md6=True, children=[m])
                    ]
                )    
            ]
        )
    ]
)
AOI_content_main